# Create datasets for training and validation

In [106]:
from numpy import inf
import pandas as pd
import numpy as np
import re
import os

## Read target feature

In [107]:
target_feature = pd.read_csv('../target_feature/01_create_target_feature.csv',
                             sep=';')

In [108]:
target_feature.head()

,folder,file_name,number_of_words,pct_pol_neg,pct_pol_pos,pct_pol_neu
0,ciclo_3,8998-Santo Antônio de Jesus-BA.pdf,45543,0.015063,0.032302,0.954087
1,ciclo_3,9024-Ulianópolis-PA.pdf,17432,0.018945,0.022160,0.959642
2,ciclo_3,9010-Aldeias Altas-MA.pdf,59605,0.022763,0.024140,0.954407
3,ciclo_3,9034-Paraíba do Sul-RJ.pdf,15486,0.014342,0.029007,0.957103
4,ciclo_3,9045-Governador Celso Ramos-SC.pdf,5177,0.011985,0.025130,0.963657


In [109]:
target_feature['temp'] = target_feature['file_name'].str.replace('[0-9]|.pdf|-', ' ', regex=True)\
    .str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower().str.strip()
target_feature['city'] = target_feature['temp'].str[:-3]
target_feature['state'] = target_feature['temp'].str[-2:]
target_feature['city_state'] = target_feature['city'].map(str) + '_' + target_feature['state']

target_feature = target_feature.drop("temp", axis=1)
target_feature.head()

,folder,file_name,number_of_words,pct_pol_neg,pct_pol_pos,pct_pol_neu,city,state,city_state
0,ciclo_3,8998-Santo Antônio de Jesus-BA.pdf,45543,0.015063,0.032302,0.954087,santo antonio de jesus,ba,santo antonio de jesus_ba
1,ciclo_3,9024-Ulianópolis-PA.pdf,17432,0.018945,0.022160,0.959642,ulianopolis,pa,ulianopolis_pa
2,ciclo_3,9010-Aldeias Altas-MA.pdf,59605,0.022763,0.024140,0.954407,aldeias altas,ma,aldeias altas_ma
3,ciclo_3,9034-Paraíba do Sul-RJ.pdf,15486,0.014342,0.029007,0.957103,paraiba do sul,rj,paraiba do sul_rj
4,ciclo_3,9045-Governador Celso Ramos-SC.pdf,5177,0.011985,0.025130,0.963657,governador celso ramos,sc,governador celso ramos_sc


## Read explanatory features: education, family, fertility and work (2000 and 2010)

In [110]:
raw_dataset = target_feature

In [111]:
state_name_to_acronym = pd.DataFrame({'full_state_name': 
                                      ['acre', 
                                       'alagoas', 
                                       'amapa', 
                                       'amazonas', 
                                       'bahia', 
                                       'ceara', 
                                       'distrito_federal', 
                                       'espirito_santo', 
                                       'goias', 
                                       'maranhao', 
                                       'mato_grosso', 
                                       'mato_grosso_do_sul', 
                                       'minas_gerais', 
                                       'para', 
                                       'paraiba', 
                                       'parana', 
                                       'pernambuco', 
                                       'piaui', 
                                       'rio_de_janeiro', 
                                       'rio_grande_do_norte', 
                                       'rio_grande_do_sul', 
                                       'rondonia', 
                                       'roraima', 
                                       'santa_catarina', 
                                       'sao_paulo', 
                                       'sergipe', 
                                       'tocantins'],
                                      'acronym': ['ac',
                                                  'al',
                                                  'ap',
                                                  'am',
                                                  'ba',
                                                  'ce',
                                                  'df',
                                                  'es',
                                                  'go',
                                                  'ma',
                                                  'mt',
                                                  'ms',
                                                  'mg',
                                                  'pa',
                                                  'pb',
                                                  'pr',
                                                  'pe',
                                                  'pi',
                                                  'rj',
                                                  'rn',
                                                  'rs',
                                                  'ro',
                                                  'rr',
                                                  'sc',
                                                  'sp',
                                                  'se',
                                                  'to']})

var_list = ['var_01',
            'var_02',
            'var_03']

In [112]:
paths = ['../ibge_censo/2000/education',
         '../ibge_censo/2000/family',
         '../ibge_censo/2000/fertility',
         '../ibge_censo/2000/work',
         '../ibge_censo/2010/education',
         '../ibge_censo/2010/family',
         '../ibge_censo/2010/fertility',
         '../ibge_censo/2010/work']

In [113]:
for path in paths:
    
    for var_name in var_list:
        full_temp = pd.DataFrame()
        
        for state in os.listdir(path):
            if not state.startswith('.'):
                state_acronym = state_name_to_acronym.loc[
                    state_name_to_acronym.full_state_name == state]['acronym'].values[0]
        
                for filename in os.listdir(path + '/' + state):
                    if not filename.startswith('.') and filename.endswith(var_name + '.csv'):
        
                        temp = pd.read_csv(path + '/' + state + '/' + filename)
                        temp['city_state'] = temp['city'].map(str) + '_' + state_acronym
                        
                        full_temp = pd.concat([full_temp, temp])
        
        if full_temp.shape[0] != 0:
            full_temp = full_temp.add_prefix(path.split("/")[2] + '_' + path.split("/")[3] + '_' + var_name + '_')
            column_to_join = path.split("/")[2] + '_' + path.split("/")[3] + '_' + var_name + '_city_state'
            
            raw_dataset = pd.merge(raw_dataset,
                                   full_temp.iloc[:,1:],
                                   left_on="city_state",
                                   right_on=column_to_join,
                                   how="left")
            
            raw_dataset = raw_dataset.drop(column_to_join, axis=1)
            
            print(path + ' [' + var_name + '] ')
        

../ibge_censo/2000/education [var_01] 
../ibge_censo/2000/family [var_01] 
../ibge_censo/2000/family [var_02] 
../ibge_censo/2000/fertility [var_01] 
../ibge_censo/2000/fertility [var_02] 
../ibge_censo/2000/fertility [var_03] 
../ibge_censo/2000/work [var_01] 
../ibge_censo/2000/work [var_02] 
../ibge_censo/2010/education [var_01] 
../ibge_censo/2010/family [var_01] 
../ibge_censo/2010/family [var_02] 
../ibge_censo/2010/fertility [var_01] 
../ibge_censo/2010/fertility [var_02] 
../ibge_censo/2010/fertility [var_03] 
../ibge_censo/2010/work [var_01] 
../ibge_censo/2010/work [var_02] 


## Read explanatory feature: social indicator (not in same pattern as others)

In [114]:
paths = ['../ibge_censo/2010/social_indicator']

### Changing city name due to city being known by two different names

In [115]:
raw_dataset.loc[raw_dataset.file_name=='3238-São Valério da Natividade-TO.pdf', 'city_state'] = 'sao valerio_to'

In [116]:
for path in paths:
    
    for var_name in var_list:
        full_temp = pd.DataFrame()
        
        for state in os.listdir(path):
            if not state.startswith('.'):
                state_acronym = state_name_to_acronym.loc[
                    state_name_to_acronym.full_state_name == state]['acronym'].values[0]
                
                for filename in os.listdir(path + '/' + state):
                    if not filename.startswith('.') and filename.endswith(var_name + '.csv'):
                        
                        temp = pd.read_csv(path + '/' + state + '/' + filename)
                        temp['city_state'] = temp['city'].map(str) + '_' + state_acronym
                        
                        full_temp = pd.concat([full_temp, temp])
                        
        if full_temp.shape[0] != 0:
            full_temp = full_temp.add_prefix(path.split("/")[3] + '_' + var_name + '_')
            column_to_join = path.split("/")[3] + '_' + var_name + '_city_state'
            
            raw_dataset = pd.merge(raw_dataset,
                                   full_temp.iloc[:,1:],
                                   left_on="city_state",
                                   right_on=column_to_join,
                                   how="left")
            
            raw_dataset = raw_dataset.drop(column_to_join, axis=1)
            
            print(path + ' [' + var_name + '] ')
        

../ibge_censo/2010/social_indicator [var_01] 
../ibge_censo/2010/social_indicator [var_02] 
../ibge_censo/2010/social_indicator [var_03] 


In [117]:
for c in raw_dataset.columns:
    print(c)

folder
file_name
number_of_words
pct_pol_neg
pct_pol_pos
pct_pol_neu
city
state
city_state
2000_education_var_01_quantity
2000_family_var_01_total
2000_family_var_01_adequada
2000_family_var_01_semi_adequada
2000_family_var_01_inadequada
2000_family_var_02_qt
2000_fertility_var_01_total
2000_fertility_var_01_has_children
2000_fertility_var_01_children_born
2000_fertility_var_01_children_borned_live
2000_fertility_var_01_children_borned_dead
2000_fertility_var_02_total
2000_fertility_var_02_married
2000_fertility_var_02_separated
2000_fertility_var_02_divorced
2000_fertility_var_02_widow
2000_fertility_var_02_single
2000_fertility_var_03_total
2000_work_var_01_total
2000_work_var_01_domestic_regular
2000_work_var_01_domestic_irregular
2000_work_var_01_other_regular
2000_work_var_01_military_and_gov
2000_work_var_01_other_irregular
2000_work_var_02_total
2000_work_var_02_regular
2000_work_var_02_military_and_gov
2000_work_var_02_irregular
2000_work_var_02_employers
2000_work_var_02_entre

In [118]:
raw_dataset.to_csv('02_data_processing_01_raw_dataset.csv',
                   sep=';',
                   index=False)

## Feature engineering

In [119]:
full_dataset = pd.read_csv('02_data_processing_01_raw_dataset.csv',
                           sep=';')

In [120]:
full_dataset = full_dataset.replace('-', 0)

full_dataset.iloc[:, 9:] = full_dataset.iloc[:, 9:].apply(pd.to_numeric)

For all the features created using IBGE, divide them from the position in 2000 by the position in 2010, with the following observations: **if the feature is not a proportion, then divide the feature by the population size of the year (2000 or 2010)**

In [121]:
full_dataset['education_var_01_qt_pct'] = (full_dataset['2000_education_var_01_quantity'] / full_dataset['2000_family_var_02_qt']) / (full_dataset['2010_education_var_01_quantity'] / full_dataset['2010_family_var_02_qt'])

full_dataset['family_var_01_adequada_pct'] = (full_dataset['2000_family_var_01_adequada'] / full_dataset['2000_family_var_01_total']) / (full_dataset['2010_family_var_01_adequada'] / full_dataset['2010_family_var_01_total'])
full_dataset['family_var_01_semi_adequada_pct'] = (full_dataset['2000_family_var_01_semi_adequada'] / full_dataset['2000_family_var_01_total']) / (full_dataset['2010_family_var_01_semi_adequada'] / full_dataset['2010_family_var_01_total'])
full_dataset['family_var_01_inadequada_pct'] = (full_dataset['2000_family_var_01_inadequada'] / full_dataset['2000_family_var_01_total']) / (full_dataset['2010_family_var_01_inadequada'] / full_dataset['2010_family_var_01_total'])

full_dataset['fertility_var_01_has_children_pct'] = (full_dataset['2000_fertility_var_01_has_children'] / full_dataset['2000_fertility_var_01_total']) / (full_dataset['2010_fertility_var_01_has_children'] / full_dataset['2010_fertility_var_01_total'])
full_dataset['fertility_var_01_children_born_pct'] = (full_dataset['2000_fertility_var_01_children_born'] / full_dataset['2000_fertility_var_01_total']) / (full_dataset['2010_fertility_var_01_children_born'] / full_dataset['2010_fertility_var_01_total'])
full_dataset['fertility_var_01_children_borned_live_pct'] = (full_dataset['2000_fertility_var_01_children_borned_live'] / full_dataset['2000_fertility_var_01_total']) / (full_dataset['2010_fertility_var_01_children_borned_live'] / full_dataset['2010_fertility_var_01_total'])
full_dataset['fertility_var_01_children_borned_dead_pct'] = (full_dataset['2000_fertility_var_01_children_borned_dead'] / full_dataset['2000_fertility_var_01_total']) / (full_dataset['2010_fertility_var_01_children_borned_dead'] / full_dataset['2010_fertility_var_01_total'])

full_dataset['fertility_var_02_married_pct'] = (full_dataset['2000_fertility_var_02_married'] / full_dataset['2000_fertility_var_02_total']) / (full_dataset['2010_fertility_var_02_married'] / full_dataset['2010_fertility_var_02_total'])
full_dataset['fertility_var_02_separated_pct'] = (full_dataset['2000_fertility_var_02_separated'] / full_dataset['2000_fertility_var_02_total']) / (full_dataset['2010_fertility_var_02_separated'] / full_dataset['2010_fertility_var_02_total'])
full_dataset['fertility_var_02_divorced_pct'] = (full_dataset['2000_fertility_var_02_divorced'] / full_dataset['2000_fertility_var_02_total']) / (full_dataset['2010_fertility_var_02_divorced'] / full_dataset['2010_fertility_var_02_total'])
full_dataset['fertility_var_02_widow_pct'] = (full_dataset['2000_fertility_var_02_widow'] / full_dataset['2000_fertility_var_02_total']) / (full_dataset['2010_fertility_var_02_widow'] / full_dataset['2010_fertility_var_02_total'])
full_dataset['fertility_var_02_single_pct'] = (full_dataset['2000_fertility_var_02_single'] / full_dataset['2000_fertility_var_02_total']) / (full_dataset['2010_fertility_var_02_single'] / full_dataset['2010_fertility_var_02_total'])

full_dataset['fertility_var_03_total_pct'] = (full_dataset['2000_fertility_var_03_total'] / full_dataset['2000_family_var_02_qt']) / (full_dataset['2010_fertility_var_03_total'] / full_dataset['2010_family_var_02_qt'])

full_dataset['work_var_01_regular_pct'] = ((full_dataset['2000_work_var_01_domestic_regular'] + full_dataset['2000_work_var_01_other_regular'] + full_dataset['2000_work_var_01_military_and_gov']) / full_dataset['2000_work_var_01_total']) / ((full_dataset['2010_work_var_01_main_regular'] + full_dataset['2010_work_var_01_other_regular']) / full_dataset['2010_work_var_01_total'])
full_dataset['work_var_01_irregular_pct'] = ((full_dataset['2000_work_var_01_domestic_irregular'] + full_dataset['2000_work_var_01_other_irregular']) / full_dataset['2000_work_var_01_total']) / ((full_dataset['2010_work_var_01_main_irregular'] + full_dataset['2010_work_var_01_other_irregular']) / full_dataset['2010_work_var_01_total'])

full_dataset['social_indicator_var_01_15_to_24_years_pct'] = (full_dataset['social_indicator_var_01_2000_15_to_24_years'] / full_dataset['social_indicator_var_01_2010_15_to_24_years'])
full_dataset['social_indicator_var_01_25_to_59_years_pct'] = (full_dataset['social_indicator_var_01_2000_25_to_59_years'] / full_dataset['social_indicator_var_01_2010_25_to_59_years'])
full_dataset['social_indicator_var_01_60_to_more_years_pct'] = (full_dataset['social_indicator_var_01_2000_60_to_more_years'] / full_dataset['social_indicator_var_01_2010_60_to_more_years'])

full_dataset['social_indicator_var_02_suitable_pct'] = full_dataset['social_indicator_var_02_2000_suitable'] / full_dataset['social_indicator_var_02_2010_suitable']
full_dataset['social_indicator_var_02_semi_suitable_pct'] = full_dataset['social_indicator_var_02_2000_semi_suitable'] / full_dataset['social_indicator_var_02_2010_semi_suitable']
full_dataset['social_indicator_var_02_inappropriate_pct'] = full_dataset['social_indicator_var_02_2000_inappropriate'] / full_dataset['social_indicator_var_02_2010_inappropriate']

full_dataset['social_indicator_var_03_responsable_illiterate_pct'] = full_dataset['social_indicator_var_03_2000_responsable_illiterate'] / full_dataset['social_indicator_var_03_2010_responsable_illiterate']
full_dataset['social_indicator_var_03_inappropriate_residence_pct'] = full_dataset['social_indicator_var_03_2000_inappropriate_residence'] / full_dataset['social_indicator_var_03_2010_inappropriate_residence']
full_dataset['social_indicator_var_03_responsable_illiterate_and_inappropriate_residence_pct'] = full_dataset['social_indicator_var_03_2000_responsable_illiterate_and_inappropriate_residence'] / full_dataset['social_indicator_var_03_2010_responsable_illiterate_and_inappropriate_residence']

### Select only generated features and remove rows with NaN values on it

In [177]:
modeling_dataset = full_dataset.iloc[:, np.r_[3:6, 7, 88:113]].dropna()

### Remove 'inf' value for all columns

In [178]:
def replace_inf_by_max(df, col_name):
    max_for_column = max(df.loc[df[col_name] != np.inf, col_name])
    df.loc[df[col_name] == np.inf, col_name] = max_for_column
    
    return df

In [179]:
for col in modeling_dataset.columns:
    modeling_dataset = replace_inf_by_max(modeling_dataset, col)

### One hot encoding for 'state' feature

In [180]:
modeling_dataset = pd.concat([
    modeling_dataset,
    pd.get_dummies(modeling_dataset['state'], prefix = 'state')
], axis = 1)

modeling_dataset = modeling_dataset.drop(columns=['state'])

In [181]:
modeling_dataset.tail()

,pct_pol_neg,pct_pol_pos,pct_pol_neu,education_var_01_qt_pct,family_var_01_adequada_pct,family_var_01_semi_adequada_pct,family_var_01_inadequada_pct,fertility_var_01_has_children_pct,fertility_var_01_children_born_pct,fertility_var_01_children_borned_live_pct,...,state_pr,state_rj,state_rn,state_ro,state_rr,state_rs,state_sc,state_se,state_sp,state_to
535,0.016456,0.030705,0.953637,1.002093,0.293259,1.029561,1.842016,0.932309,1.203601,1.176372,...,0,0,0,0,0,0,0,0,0,0
536,0.019006,0.031450,0.952185,0.996090,0.839330,1.067334,3.925634,0.928454,1.245205,1.237379,...,0,0,0,0,0,0,0,0,0,0
537,0.021571,0.025940,0.954154,1.080277,0.000000,0.779659,3.520802,1.016201,1.416888,1.339411,...,0,0,0,0,0,0,0,0,0,0
538,0.016244,0.026293,0.958579,0.759617,0.636611,0.870724,4.013250,0.944821,1.236316,1.182111,...,1,0,0,0,0,0,0,0,0,0
539,0.012584,0.027566,0.962011,0.929166,0.913296,1.166338,12.687498,0.978698,1.098397,1.109735,...,0,0,0,0,0,0,0,0,1,0


In [185]:
modeling_dataset.to_csv('02_data_processing_02_modeling_dataset.csv',
                        sep=';',
                        index=False)

## Splitting the datasets (training and validation)

In [186]:
training_dataset = modeling_dataset.sample(frac=0.75,
                                           random_state=7)
validation_dataset = modeling_dataset.drop(training_dataset.index)

In [187]:
print('=== Number of rows === \n' +
      'Training: ' + str(len(training_dataset)) + '\n' +
      'Validation: ' + str(len(validation_dataset)))

=== Number of rows === 
Training: 382
Validation: 127


In [189]:
training_dataset.head(15)

,pct_pol_neg,pct_pol_pos,pct_pol_neu,education_var_01_qt_pct,family_var_01_adequada_pct,family_var_01_semi_adequada_pct,family_var_01_inadequada_pct,fertility_var_01_has_children_pct,fertility_var_01_children_born_pct,fertility_var_01_children_borned_live_pct,...,state_pr,state_rj,state_rn,state_ro,state_rr,state_rs,state_sc,state_se,state_sp,state_to
381,0.018576,0.029370,0.953309,0.975146,0.771346,1.100852,3.460448,0.991619,1.116080,1.096923,...,0,0,0,0,0,0,0,0,1,0
357,0.014563,0.028014,0.959195,0.977967,0.156153,1.301859,8.095948,0.899278,1.155976,1.152184,...,0,0,0,0,0,0,0,0,0,0
312,0.023637,0.035859,0.943328,1.045255,0.813104,1.349589,4.570161,0.957922,1.069060,1.070987,...,0,0,0,0,0,0,0,0,1,0
13,0.013638,0.026332,0.960974,1.023528,0.053283,0.898975,2.613556,0.976104,1.156115,1.157691,...,0,0,0,0,0,0,0,0,0,0
447,0.016828,0.045385,0.939317,0.785903,0.499677,1.276335,8.326532,1.060698,1.136891,1.112883,...,0,0,0,0,0,0,0,0,1,0
431,0.017115,0.026508,0.958124,0.940642,0.242616,0.863668,2.789767,0.973271,1.013384,1.024474,...,0,0,0,1,0,0,0,0,0,0
512,0.015075,0.028007,0.958604,1.169528,1.096991,0.905359,3.529700,0.996485,1.191395,1.195416,...,0,0,0,0,0,0,1,0,0,0
179,0.021466,0.029777,0.951264,0.911090,0.146323,0.930814,2.914887,0.985468,1.176389,1.197157,...,0,0,0,0,0,0,0,0,0,0
69,0.018447,0.029067,0.954309,0.981606,1.534059,0.850321,1.943696,0.955209,1.096616,1.051998,...,0,0,0,0,0,0,0,0,0,0
498,0.015602,0.024583,0.961266,1.039032,1.462733,0.835988,5.118509,0.978452,1.092314,1.087017,...,0,0,0,0,0,1,0,0,0,0


In [190]:
training_dataset.to_csv('02_data_processing_03_training_dataset.csv',
                        sep=';',
                        index=False)

validation_dataset.to_csv('02_data_processing_04_validation_dataset.csv',
                          sep=';',
                          index=False)